#### Modelling the sequential FPL team selection process as a Belief-State Markov Decision Process
- For the $i$-th gameweek, we define the following terms:
    - $M_i$ is set of matches in gameweek $i$.
    - $P_i$ is the set of players available for selection in gameweek $i$.
    - $A_i$ is the set of actions available in gameweek $i$, where $a \in A_i$ is a subset of $P_i$ and observes all team selection constraints.
    - $p_i \in P_i$ is associated with its FPL-designated position $pos(p_i)$ and price $pr(p_i)$.
    - $\tau_p \in \tau$ is a system of distributions representing player's performance/influence on the matchplay.
    - $O_i$ is the set of match observations in gameweek $i$
    - $o \in O_i$ includes both the result of the matches and the performance of the players in the selected team e.g. goals, assists, clean sheets, yellow cards, red cards, bonus points. The probability of each $o \in O_i$ is somehow dependent on the players' characteristics ($\tau$) i.e. a team with strong attackers is more likely to score goals, therefore, $P(o | \tau)$ is dependent on $\tau$.
    - $R(o, a_{prev}, a_{curr})$ is the reward function, which returns the points scored by the selected team $a_{curr}$, given the match observations $o$. The previous team $a_{prev}$ is also provided to penalize the agent for any player poor player transfers or transfers beyond the allowed number.

Markov's Decision Process (MDP) 
- A state $S_i$ would encapsulate
    - $M_{i,..., 38}$ - set of upcoming fixtures for that gameweek
    - $P_i$ - set of players available for selection
    - $o \in O_{i - 1}$ - the outcome of the previous gameweek
    - $\tau$ - the system of distributions representing players' abilities
- An action $A_i$ is the set of teams selectable in gameweek $i$
- $R$ is the corresponding reward function

Belief model ($\tau$):
- Represent uncertainty over players' abilities and generate samples $\tau$ from the distribution $Pr(\tau | b)$.
- Three distributions are used to model the players' abilities:
    - $\rho_p$ - a three-state categorical distribution representing the player's probability of starting a match, being substituted, or not playing at all i.e. (start, sub, unused).
    - $\omega_p$ - a Bernoulli/Binomial distribution over a single trial, representing the probability of a player scoring a goal given he was playiong at the time
    - $\psi_p$ - a Bernoulli distribution representing the probability of a player providing an assist given he was playing at the time
- Define prior distributions over the parameters of the above distributions and update them using the match observations $o$ to obtain new posterior distributions.
- Use simple closed-form equations e.g. Beta and Dirichlet conjugate priors to update the priors.
- Sample from these conjugate distributions to generate $\tau_p$.
- Define hyperparemeters uniformly across all players i.e. $$\omega_p \sim Beta(1, 1), \psi_p \sim Beta(1, 1),  \rho_p \sim Dirichlet(\frac{1}{4}, \frac{1}{4}, \frac{1}{4})$$
- Potential to use performance data from previous seasons to define priors
- Define 4 global multinomial distributions $S_{pos}$ - one for each position - to describe the distribution of minutes players who play the same position $pos$ are likely to play in a match, given they start the match.
- Player absence via injury/suspension or any other reson is modelled by setting the probability of starting and substituting to zero i.e. $Pr(\rho_p = start) \text{and} Pr(\rho_p = sub) = 0$.

In [1]:
from utils.data_registry import DataRegistry
import pymc as pm
import numpy as np
import pandas as pd

# There are 38 gameweeks in a season
GAMEWEEK_COUNT = 38

seasonal_gameweek_player_data = DataRegistry()

Standardized gameweek data saved to csv!


In [ ]:
# Define player ability (only for players in the 2023/24 season) priors to reflect the occurrences of previous seasons i.e. 2016/17 - 2022/23
player_ability_df = pd.DataFrame()
player_ability_df["name"] = seasonal_gameweek_player_data.player_data["2023-24"]["first_name"] + " " + seasonal_gameweek_player_data.player_data["2023-24"]["second_name"]
player_ability_df["ρ_β"] = [np.array([1/4, 1/4, 1/4])] * len(player_ability_df) # Dirichlet prior for ρ
player_ability_df[("ω","α")] = np.ones(len(player_ability_df)) # Beta prior for ω
player_ability_df[("ω","β")] = np.ones(len(player_ability_df)) # Beta prior for ω
player_ability_df[("ψ","α")] = np.ones(len(player_ability_df)) # Beta prior for ψ
player_ability_df[("ψ","β")] = np.ones(len(player_ability_df)) # Beta prior for ψ

for season, gameweek_data_df in seasonal_gameweek_player_data.gameweek_data.items():
    for gameweek_count in range(1, GAMEWEEK_COUNT + 1):
        for player_name in player_ability_df["name"]:
            """ # Longest common subsequence 
                - Check how often it occurs
            """
            player_gameweek_data = gameweek_data_df.loc[(gameweek_data_df["GW"] == gameweek_count) & (gameweek_data_df["name"] == player_name)]
            if not player_gameweek_data.empty:
                with pm.Model() as sequential_player_ability_model:
                    # Retrieve respective player's priors
                    ρ_β = player_ability_df.loc[player_ability_df["name"] == player_name]["ρ_β"]
                    ω_α = player_ability_df.loc[player_ability_df["name"] == player_name]["ω"]["α"]
                    ω_β = player_ability_df.loc[player_ability_df["name"] == player_name]["ω"]["β"]
                    ψ_α = player_ability_df.loc[player_ability_df["name"] == player_name]["ψ"]["α"]
                    ψ_β = player_ability_df.loc[player_ability_df["name"] == player_name]["ψ"]["β"]

                    # Priors for the current iteration
                    ρ = pm.Dirichlet('ρ', a=ρ_β)
                    ω = pm.Beta('ω', alpha=ω_α, beta=ω_β)
                    ψ = pm.Beta('ψ', alpha=ψ_α, beta=ψ_β)

                    # ρ Likelihood (multinomial)
                    ρ_observed = np.array([])





In [ ]:
# Global multinomial distributions for players' minutes played at each position
player_positions = player_data_2023_2024["element_type"].unique()

with pm.Model() as player_position_minutes_played_model:
    # 91 possible minutes in a match (0-90)
    α = np.ones(91)
    # Dirichlet prior for minutes played at each position
    gk = pm.Dirichlet('gk', a=α, shape=91)
    df = pm.Dirichlet('def', a=α, shape=91)
    mid = pm.Dirichlet('mid', a=α, shape=91)
    fwd = pm.Dirichlet('fwd', a=α, shape=91)
    

    